In [1]:
import numpy as np

def relu(x):
    '''
    return output of Rectified Linear Unit and derivative
    '''
    f=np.maximum(0,x)
    df=np.where(x>0,1,0)
    return f,df
def sigmoid(x):

    f=1/(1+np.exp(-x))
    df=f*(1-f)
    return f,df
def tanh(x):
    f=-1+2/(1+np.exp(-2*x))
    df=1-f**2
    return f,df
def identity(x):
    '''
    恒等函数
    '''
    f=x
    df=1
    return f,df
def selu(x):
    alpha=1.67326
    lambd=1.05070
    f=lambd*np.where(x>=0,x,alpha*(np.exp(x)-1))
    df=lambd*np.where(x>=0,1,alpha*np.exp(x))
    return f,df
activation_table={
    'relu':relu,
    'sigmoid':sigmoid,
    'tanh':tanh,
    'identity':identity,
    'selu':selu
}
#loss function
def squared_error(y,yhat):
    '''
    input target predict
    return mean squared error
    '''
    return np.sum(0.5*(y-yhat)**2),y-yhat
def identity_loss(y,yhat):
    return yhat,yhat

loss_table={
    'squared_error':squared_error,
    'identity':identity_loss
}


In [ ]:
class NeuralNetwork:
    def __init__(self,layer_dimensions,parameters):
        #init the neuralnetwork weights
        self.weights={}
        for i in range(len(layer_dimensions)-1):
            self.weights[i]=np.random.uniform(-0.1,0.1,
                                            (layer_dimensions[i],layer_dimensions[i+1]))
            
        self.learning_rate=parameters['learning_rate']
        self.num_iteration=parameters['num_iteration']
        self.batch_size=parameters['batch_size']

        activation_name=parameters['activation']
        if isinstance(activation_name,str) and activation_name in activation_table:
            self.activation=activation_table[activation_name]
        else:
            self.activation=activation_name
        loss_name=parameters['loss']
        if isinstance(loss_name,str) and loss_name in loss_table:
            self.loss=loss_table['loss_name']
        else:
            self.loss=loss_name
    
    def feedforward(self,x):
        #feed forward
        self.layer_input={}
        #the output of the previous layer and the input of the next layer
        self.layer_output={0:x}
        for i in range(len(self.weights)):
            self.layer_input[i]=np.dot(self.layer_output[i],self.weights[i])
            self.layer_output[i+1]=self.activation(self.layer_input[i])[0]
        return self.layer_output[len(self.weights)]
    #input is calculated by the inner product of the output of i and weight
    #output is calculated by activating the input 
    def backpropagation(self,y,yhat):
        num_layers=len(self.weights)
        #the output of last layer is predict
        delta=-1*self.loss(y,yhat)[1]*self.activation(self.layer_input[num_layers-1])[1]
        #initialize the last layer gradient weights
        gradient_weights={num_layers-1:np.dot(self.layer_output[num_layers-1].T,delta)}
        for i in reversed(range(num_layers-1)):
            delta=np.dot(delta,self.weights[i+1].T)*self.activation(self.layer_input[i])[1]
            gradient_weights[i]=np.dot(self.layer_output[i].T,delta)
        return gradient_weights
    